In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import vdmlab as vdm

In [ ]:
count_matrix = np.array([[3., 2., 1.],
                         [0., 0., 0.],
                         [1., 0., 2.],
                         [0., 2., 0.]])

In [ ]:
activity_matrix = bool_counts(count_matrix)
prob_active = prob_active_neuron(activity_matrix)
prob_expected = expected_cooccur(prob_active)
prob_observed = observed_cooccur(activity_matrix)
prob_shuffle = shuffle_cooccur(activity_matrix, num_shuffles=100)
# prob_zscore = zscore_cooccur(prob_observed, prob_shuffle)

In [ ]:
%time
prob_zscore = zscore_cooccur(prob_observed, prob_shuffle)

In [ ]:
prob_zscore

In [ ]:
linear = dict()
linear['position'] = np.concatenate((np.linspace(0, 10, 10), np.linspace(10, 0, 10), np.linspace(0, 10, 20)))
linear['time'] = np.linspace(0, 100, 40)

In [ ]:
plt.plot(linear['time'], linear['position'])
plt.show()

In [ ]:
import random
random.seed(0)
spikes = dict(time=[[], [], [], []])
for i in range(10):
    spikes['time'][0].append(random.uniform(0, 100))
for i in range(15):
    spikes['time'][1].append(random.uniform(20, 30))
for i in range(15):
    spikes['time'][2].append(random.uniform(60, 80))   
for i in range(15):
    spikes['time'][3].append(random.uniform(5, 10))
for i in range(5):
    for neuron_spikes in spikes['time']:
        neuron_spikes.append(random.uniform(0, 100))

In [ ]:
for idx, neuron_spikes in enumerate(spikes['time']):
    plt.plot(neuron_spikes, np.ones(len(neuron_spikes))+idx, '|', color='k', ms=10, mew=1)
plt.ylim(0.5, 4.5)
plt.show()

In [ ]:
tuning = vdm.tuning_curve(linear, spikes, sampling_rate=1, binsize=3, filter_type=None)

In [ ]:
colours = ['b', 'g', 'r', 'c']

x = list(range(0, np.shape(tuning)[1]))
for i, neuron_tc in enumerate(tuning):
    plt.plot(neuron_tc, color=colours[i])
#     plt.fill_between(x, 0, neuron_tc, facecolor=colours[i])
plt.show()

In [ ]:
tuning